## Transfer CLAMP TXT results to dataframe

In [1]:
import numpy as np
import pandas as pd

import re
# import re
from os.path import join
import os
import glob

from tools import *

input_prefix="/data/liu/mimic3/CLAMP_NER/input"
output_prefix="/data/liu/mimic3/CLAMP_NER/ner-attribute/output"
save_csv_prefix="/data/liu/mimic3/CLAMP_NER/ner-attribute/output_csv"
mapping_prefix = "/data/liu/mimic3/MAPPING/"
processed_map_prefix="/data/liu/mimic3/MAPPING/PROCESSED/"

In [2]:
## Import CLAMP NER result
def splitfilename(filename):
    ids= filename.rsplit(".",1)[0].split("_")
    return ids

def readtxt(filename):
    text=""
    with open(filename,'r') as myfile:
        text=myfile.read()
    return text


# def files2df(folder):
#     os.chdir(join(output_prefix,folder))
#     output_data = [getfileinfo(f) for f in glob.glob("*.txt") if (os.path.isfile(f))]
#     return output_data
                                                   
def parseNErow(row):

    indexes=row[1:3]
    attrs=[attr.split('=')[1] for attr in row[3:] if ("=" in attr)]
    return indexes+attrs

# def parseNEtext(text,num_cols=8, 
# #                 cols=["START INDEX","END INDEX","SEMANTIC","ASSERTION","CUI","NE"]
#                cols=["START INDEX","END INDEX","SEMANTIC","ASSERTION","CUI","sentProb","conceptProb","NE"]):
#     df = pd.DataFrame([parseNErow(x.split('\t')) for x in text.split('\n')])
#     if(len(df.columns)==num_cols): 
#         df.columns=cols
#         return df
#     else: return None
    
def parseNEtext(text,
#                 num_cols=8, 
#                 cols=["START INDEX","END INDEX","SEMANTIC","ASSERTION","CUI","NE"]
#                cols=["START INDEX","END INDEX","SEMANTIC","ASSERTION","CUI","sentProb","conceptProb","NE"]
               ):
    df = pd.DataFrame([x.split('\t') for x in text.split('\n')])
    return df
#     if(len(df.columns)==num_cols): 
#         df.columns=cols
#         return df
#     else: return None

def savetext2csv(folder, write=True):
    os.chdir(join(output_prefix,folder))
    files = [f for f in glob.glob("*.txt") if (os.path.isfile(f))]

    li=[]
    for file in files:
        text=readtxt(file)

        df = parseNEtext(text.split("\nSentence",1)[0])
        if(not df is None):
            li.append((file, df))
#             if(write):
#                 filename=file.replace(".txt","")
#                 write2file(df,join(save_csv_prefix,folder,filename))
    return li


def addidcolumns(file, df):
    idcols=['SUBJECT_ID','HADM_ID','GROUP_RANK']
    ids = splitfilename(file)
    for i in range(3):
        df.insert(i, idcols[i], ids[i])
    return df

def concatcsvs(folder,write=False):
    csvdf = savetext2csv(folder,write)
    concatcsv = [addidcolumns(file, df) for file, df in csvdf]
    concateddf = pd.concat(concatcsv, axis=0)
    if(write):
        write2file(concateddf,join(save_csv_prefix,folder,"ALL/ALL"))
    return concateddf
    

In [2]:
for folder in ["hospital_course","medications_on_admission","medical_history"]:
    concatcsv = concatcsvs(folder,True)

NameError: name 'concatcsvs' is not defined

In [3]:
aa = ['HOSPITAL COURSE', 'MEDICAL HISTORY', 'MEDICATIONS ON ADMISSION']
[a.replace(" ","_") for a in aa]

['HOSPITAL_COURSE', 'MEDICAL_HISTORY', 'MEDICATIONS_ON_ADMISSION']

In [2]:
for folder in ["hospital_course","medications_on_admission","medical_history"]:
    concatcsv = read_data(join(save_csv_prefix,folder,"ALL/ALL"))
    print("{:}: {:,d} episodes".format(folder,len(concatcsv.HADM_ID.unique())))

/home/liu/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


hospital_course: 15,289 episodes


/home/liu/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


medications_on_admission: 15,289 episodes
medical_history: 15,289 episodes


## Extract Drugs from Clamp Results

In [4]:
folder = "hospital_course"
concatcsv = read_data(join(save_csv_prefix,folder,"ALL/ALL"))
concatcsv.head()

/home/liu/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,SUBJECT_ID,HADM_ID,GROUP_RANK,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,19981,159283,2,NamedEntity,24.0,29.0,semantic=test,assertion=present,cui=C0005767,sentProb=0.417,conceptProb=0.831,ne=BLOOD,NaN,NaN,NaN,NaN,NaN
1,19981,159283,2,NamedEntity,30.0,33.0,semantic=test,assertion=present,cui=C0023516,sentProb=0.417,conceptProb=0.589,ne=WBC,NaN,NaN,NaN,NaN,NaN
2,19981,159283,2,NamedEntity,34.0,38.0,semantic=labvalue,ne=14.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19981,159283,2,NamedEntity,41.0,44.0,semantic=test,assertion=present,cui=C0014792,sentProb=0.417,conceptProb=1.0,ne=RBC,NaN,NaN,NaN,NaN,NaN
4,19981,159283,2,NamedEntity,45.0,49.0,semantic=labvalue,ne=3.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df['3'].unique()

array(['semantic=test', 'semantic=labvalue', 'semantic=drug',
       'semantic=drug::STRENGTH', 'drug', 'labvalue', 'semantic=problem',
       'semantic=BDL', 'semantic=treatment', 'problem', 'semantic=NEG',
       'semantic=SEV', 'semantic=UNC', 'semantic=COU',
       'section=NONEHEADER', 'pos=NN', 'pos=.', nan, 'semantic=CON',
       'semantic=drug::RUT', 'semantic=SUB', 'pos=JJ', 'pos=IN',
       'pos=HYPH', 'semantic=DBN-DPN-DIN', 'semantic=drug::FREQ',
       'pos=CD', 'section=references', 'pos=VB', 'pos=:',
       'semantic=drug::DRT', 'pos=DT', 'semantic=drug::FORM', 'pos=,',
       'pos=VBD', 'semantic=UNIT', 'pos=NNS', 'semantic=DDF-DOSEUNIT-RUT',
       'semantic=drug::NEC', 'pos=RB', 'pos=SYM',
       'semantic=drug::DOSEAMT', 'pos=VBN', 'pos=PRP', 'pos=VBZ',
       'pos=NNP', 'pos=CC', 'semantic=DDF-DIN', 'semantic=DOSEUNIT-DIN',
       'pos=WRB', 'pos=LS', 'semantic=RUT-DPN-DIN',
       'section=family_history', 'pos=PRP$', 'section=history_source',
       'pos=MD'], dty

In [1]:
a={1:"sdf",2:"fsddsf",3:"fdssd"}
for i, k in a.items():
    print(i)
    print(k)

1
sdf
2
fsddsf
3
fdssd


In [6]:
a[1].endswith("f")

True

In [5]:
df=concatcsv
drugdf = df[(df['3']=="semantic=drug")]
# & (df['4'].str.contains("RxNorm")).iloc[:,0:11]
# drugdf.head(30)

In [5]:
def clean_value(value):

    if isinstance(value, str):
        if value.find("="):
            return re.sub('[\[\]]', '', value.split("=")[-1])
#         else: return np.nan
#     else:
#         return None
#     return re.sub('[\[\]]', '', value.split("=")[-1])

def getdrug(df):
    drugdf = df[(df['3']=="semantic=drug") & (df['4'].str.contains("RxNorm"))].drop(columns=['0']).iloc[:,0:7]
    cols=["SUBJECT_ID", "HADM_ID", "GROUP_RANK","START INDEX","END INDEX","SEMANTIC","CUI"]
    drugdf.columns=cols
    drugdf = drugdf.reset_index()
    
    
    drugdf["CUI"], drugdf["RxNorm"], drugdf["Generic"] = drugdf.CUI.str.split(",").str
#     drugdf=drugdf.dropna(subset=drugdf.columns[6:8])
#     drugdf.iloc[:,87:9]=drugdf.iloc[:,7:9].apply(clean_value)
    for col in drugdf.columns[6:]:
        drugdf[col].astype(str)
        drugdf[col]=drugdf[col].apply(clean_value)

#     drugdf["RxNorm"]=drugdf["RxNorm"].apply(lambda x: x[x.find("[")+1:-1])
#     drugdf["Generic"]=drugdf["Generic"].apply(lambda x: x[x.find("[")+1:-1])

    return drugdf

In [6]:
for folder in ["hospital_course","medical_history","medications_on_admission"]:
    concatcsv = read_data(join(save_csv_prefix,folder,"ALL/ALL"))
    drug_df = getdrug(concatcsv)
    write2file(drug_df,join(save_csv_prefix,folder,"ALL/ALL_Drug"))

/home/liu/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [7]:
drug_df.head()

,index,SUBJECT_ID,HADM_ID,GROUP_RANK,START INDEX,END INDEX,SEMANTIC,CUI,RxNorm,Generic
0,11,19981,159283,2,478.0,481.0,drug,1025540,1025540,None
1,58,6702,143782,1,442.0,447.0,drug,C1599049,566621,202991
2,59,6702,143782,1,459.0,464.0,drug,7447,7447,None
3,70,6702,143782,1,880.0,890.0,drug,C0042313,11124,11124
4,71,6702,143782,1,892.0,904.0,drug,C0282386,82122,82122


In [2]:
## drug rxnorm df
folder="hospital_course"
drug_df = read_data(join(save_csv_prefix,folder,"ALL/ALL_Drug"),dtype={"RxNorm":int})
#     write2file(drug_df,join(save_csv_prefix,folder,"ALL/ALL_Drug"))
drug_df.head()

,index,SUBJECT_ID,HADM_ID,GROUP_RANK,START INDEX,END INDEX,SEMANTIC,CUI,RxNorm,Generic
0,55,19981,159283,2,447.0,450.0,drug,C0106269,47104,47104.0
1,73,19981,159283,2,548.0,551.0,drug,C0106269,47104,47104.0
2,88,19981,159283,2,640.0,647.0,drug,C0002712,743,743.0
3,92,19981,159283,2,695.0,701.0,drug,C0023764,6406,6406.0
4,94,19981,159283,2,736.0,743.0,drug,C0006675,1895,1895.0


In [3]:
norm_ndc=read_data(join(processed_map_prefix,"rxnorm_ndc_list_df"),dtype={"CUI_CODE":int,"NDCs":str})
norm_ndc.head()

,CUI_CODE,CODE
0,1000000,"65597011530,65597011590,65597011507,6559701151..."
1,1000001,"00093500456,00093500498,13668038505,1366803853..."
2,1000005,"65597011630,65597011690,65597011607,6559701161..."
3,1000006,"00024131302,00024131326,00074121201,0007412130..."
4,1000013,"22840575602,22840575605,54575047602,5457504761..."


In [4]:
 norm_ndc.CUI_CODE.min()

1952

In [5]:
norm_ndc_map = norm_ndc.set_index('CUI_CODE')['CODE'].to_dict()

In [9]:
for folder in ["hospital_course","medical_history","medications_on_admission"]:
#     concatcsv = read_data(join(save_csv_prefix,folder,"ALL/ALL"))
#     drug_df = getdrug(concatcsv)
    
# ["hospital_course","medications_on_admission","medical_history"]:
    drug_df = read_data(join(save_csv_prefix,folder,"ALL/ALL_Drug"),dtype={"RxNorm":int}).drop(columns=["index"])
    drug_df["NDCs"]=drug_df["RxNorm"].apply(lambda x: norm_ndc_map.get(x, None))
    write2file(drug_df,join(save_csv_prefix,folder,"ALL/ALL_Drug_NDCs"))
#     write2file(disease_df,join(save_csv_prefix,folder,"ALL/ALL_Disease_icd9"))

In [8]:
drug_df[drug_df['NDCs'].notna()]

,SUBJECT_ID,HADM_ID,GROUP_RANK,START INDEX,END INDEX,SEMANTIC,CUI,RxNorm,Generic,NDCs
109,14617,132697,1,3856.0,3859.0,drug,NaN,1010895,1010895.0,"00363317803,00363317806,00363451047,0036345107..."
110,14617,132697,1,3920.0,3923.0,drug,NaN,1010895,1010895.0,"00363317803,00363317806,00363451047,0036345107..."
111,14617,132697,1,4235.0,4238.0,drug,NaN,1010895,1010895.0,"00363317803,00363317806,00363451047,0036345107..."
112,14617,132697,1,4323.0,4326.0,drug,NaN,1010895,1010895.0,"00363317803,00363317806,00363451047,0036345107..."
114,14617,132697,1,4564.0,4567.0,drug,NaN,1010895,1010895.0,"00363317803,00363317806,00363451047,0036345107..."
484,371,113500,1,35.0,38.0,drug,198040,198040,NaN,"43598043701,47781045801,59762048901,6042908410..."
685,47380,176418,1,1182.0,1196.0,drug,237205,237205,NaN,"00074410401,00074410701,00517480510,0051748101..."
743,23063,171508,1,618.0,621.0,drug,198040,198040,NaN,"43598043701,47781045801,59762048901,6042908410..."
966,30463,195554,1,86.0,89.0,drug,NaN,1010895,1010895.0,"00363317803,00363317806,00363451047,0036345107..."
1019,3476,189018,1,4.0,14.0,drug,C0701677,861008,151827.0,"00087606005,00087606010,00247143730,0024714433..."


In [11]:
drug_df[drug_df['NDCs'].notnull()]

,SUBJECT_ID,HADM_ID,GROUP_RANK,START INDEX,END INDEX,SEMANTIC,CUI,RxNorm,Generic,NDCs


In [12]:
drug_df[drug_df['RxNorm'].str.contains('743')]

array([None], dtype=object)

## Extract diseases from CLAMP results

In [8]:
def getdisease(df):
    diseasedf = df[(df['3']=="semantic=problem") & (df['5'].str.contains("SNOMEDCT_US"))].drop(columns=['0']).iloc[:,0:11]
    cols=["SUBJECT_ID", "HADM_ID", "GROUP_RANK","START INDEX","END INDEX","SEMANTIC","ASSERTION","CUI","sentProb","conceptProb","NE"]
    diseasedf.columns=cols
    diseasedf = diseasedf.reset_index()
    
    diseasedf["CUI"], diseasedf["SNOMEDCT_US"] = diseasedf.CUI.str.split(" ",1).str
    diseasedf=diseasedf.dropna(subset=["SNOMEDCT_US"])
    diseasedf["SNOMEDCT_US"]=diseasedf["SNOMEDCT_US"].apply(lambda x: x[x.find("[")+1:-1])
    return diseasedf

In [13]:
for folder in ["hospital_course"]:
    concatcsv = concatcsvs(folder,False)
    disease_df = getdisease(concatcsv)
    write2file(disease_df,join(save_csv_prefix,folder,"ALL/ALL_Disease"))

NameError: name 'concatcsvs' is not defined

In [102]:
disease_df.head()

,index,SUBJECT_ID,HADM_ID,GROUP_RANK,START INDEX,END INDEX,SEMANTIC,ASSERTION,CUI,sentProb,conceptProb,NE,SNOMEDCT_US
0,4,6702,143782,1,111,127,semantic=problem,assertion=present,cui=C0032227,sentProb=0.825,conceptProb=1.0,ne=pleural effusion,60046008
1,7,6702,143782,1,172,186,semantic=problem,assertion=present,cui=C0001418,sentProb=0.524,conceptProb=0.996,ne=adenocarcinoma,"443961001, 35917007"
2,21,20630,161386,1,211,222,semantic=problem,assertion=absent,cui=C0702249,sentProb=0.992,conceptProb=0.999,ne=infiltrates,231287002
3,22,20630,161386,1,226,235,semantic=problem,assertion=absent,cui=C0013687,sentProb=0.992,conceptProb=0.993,ne=effusions,41699000
4,3,17496,127002,1,105,113,semantic=problem,assertion=present,cui=C0277797,sentProb=0.677,conceptProb=0.678,ne=afebrile,86699002


In [16]:
# concatcsv.SEMANTIC.unique()
concatcsv=read_data(join(save_csv_prefix,"medications_on_admission/ALL/ALL"))

In [17]:
concatcsv.SEMANTIC.unique()

array(['treatment', 'drug', 'problem', 'BDL', 'NEG', 'test', 'BDL_Of',
       'NEG_Of', 'COU', 'drug::STRENGTH', 'drug::RUT', 'COU_Of', 'attrOf',
       'labvalue', 'valueOf', 'UNC', 'drug::FREQ', 'SEV', 'drug::NEC',
       'drug::DRT', 'UNC_Of', 'SEV_Of', 'drug::FORM', 'CON', 'CON_Of',
       'SUB', 'SUB_Of', 'DDF-DIN', 'DDF-DOSEUNIT-RUT', 'drug::DOSEAMT',
       'DOSEUNIT-DIN', 'DDF-DOSEUNIT-DPN', 'DBN-DPN-DIN', 'RUT-DPN-DIN',
       'DDF-DOSEUNIT-DIN', 'DPN-DIN'], dtype=object)

In [14]:
concatcsv.SEMANTIC.unique()

array(['treatment', 'drug', 'problem', 'BDL', 'NEG', 'test', 'BDL_Of',
       'NEG_Of', 'COU', 'drug::STRENGTH', 'drug::RUT', 'COU_Of', 'attrOf',
       'labvalue', 'valueOf', 'UNC', 'drug::FREQ', 'SEV', 'drug::NEC',
       'drug::DRT', 'UNC_Of', 'SEV_Of', 'drug::FORM', 'CON', 'CON_Of',
       'SUB', 'SUB_Of', 'DDF-DIN', 'DDF-DOSEUNIT-RUT', 'drug::DOSEAMT',
       'DOSEUNIT-DIN', 'DDF-DOSEUNIT-DPN', 'DBN-DPN-DIN', 'RUT-DPN-DIN',
       'DDF-DOSEUNIT-DIN', 'DPN-DIN'], dtype=object)

## Create Folders for NER ATTR

In [1]:
## create folder
import os

# define the name of the directory to be created
prefix = "/data/liu/mimic3/CLAMP_NER/ner-attribute"
output=join(prefix,"output")
output_csv=join(prefix,"output_csv")

def create_folder(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
        
create_folder(output)
create_folder(output_csv)
for subfolder in ["hospital_course","medical_history","medications_on_admission"]:
    create_folder(join(output,subfolder))
    create_folder(join(output_csv,subfolder))
#     create_folder(join(output_csv,subfolder,"ALL"))
    
## if want to delete folder
# import shutil
# shutil.rmtree("path_to_dir")

## RXNORM CUI -> NDC

In [8]:
rxnorm_ndc_df = pd.read_csv(join(mapping_prefix,"RXNSAT.RRF"),sep="|",header=None)
rxnorm_ndc_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,3,NaN,NaN,8717795,AUI,58488005,NaN,NaN,UMLSAUI,RXNORM,A27769867,NaN,NaN,NaN
1,3,NaN,NaN,8717795,AUI,58488005,NaN,NaN,UMLSCUI,RXNORM,C0000052,NaN,NaN,NaN
2,3,NaN,NaN,8717796,AUI,58488005,NaN,NaN,UMLSAUI,RXNORM,A27780666,NaN,NaN,NaN
3,3,NaN,NaN,8717796,AUI,58488005,NaN,NaN,UMLSCUI,RXNORM,C0000052,NaN,NaN,NaN
4,3,NaN,NaN,8717808,AUI,58488005,NaN,NaN,UMLSAUI,RXNORM,A27766654,NaN,NaN,NaN


In [9]:
rxnorm_ndc_df=rxnorm_ndc_df.iloc[:,[0,3,4,5,8,9,10]]
cols=["CUI_CODE","SAB_CODE","CAT","CUI_SUBCODE","CODE_TYPE","SOURCE","CODE"]
rxnorm_ndc_df.columns = cols
rxnorm_ndc_df.head()

,CUI_CODE,SAB_CODE,CAT,CUI_SUBCODE,CODE_TYPE,SOURCE,CODE
0,3,8717795,AUI,58488005,UMLSAUI,RXNORM,A27769867
1,3,8717795,AUI,58488005,UMLSCUI,RXNORM,C0000052
2,3,8717796,AUI,58488005,UMLSAUI,RXNORM,A27780666
3,3,8717796,AUI,58488005,UMLSCUI,RXNORM,C0000052
4,3,8717808,AUI,58488005,UMLSAUI,RXNORM,A27766654


In [7]:
for cui_code in drug_df.head()['RxNorm']:
    print(rxnorm_ndc_df[(rxnorm_ndc_df['CUI_CODE']==int(cui_code))]['CODE'][:5])

Series([], Name: CODE, dtype: object)
Series([], Name: CODE, dtype: object)
42231    NLM (1966)
42232       T002069
42233      A0023196
42234      C0002712
42235    EC 3.2.1.-
Name: CODE, dtype: object
223976    NLM (1966)
223977       T023981
223978      A0080311
223979      C0023764
223980    EC 3.1.1.3
Name: CODE, dtype: object
79192          A0033966
79193          C0006675
79194    POPLINE (1984)
79195    FDA SRS (2014)
79196           T006026
Name: CODE, dtype: object


In [19]:
# rxnorm_ndc_df[(rxnorm_ndc_df['SOURCE']=="RXNORM") & (rxnorm_ndc_df['CODE_TYPE']=="NDC")]

In [10]:
rxnorm_ndc_df = rxnorm_ndc_df[(rxnorm_ndc_df["CODE_TYPE"]=="NDC") & (rxnorm_ndc_df["CODE"].str.len()==11)]
rxnorm_ndc_df.head()

,CUI_CODE,SAB_CODE,CAT,CUI_SUBCODE,CODE_TYPE,SOURCE,CODE
83838,1952,9755475,AUI,006104,NDC,NDDF,17317093201
83839,1952,9755475,AUI,006104,NDC,NDDF,49452162501
83840,1952,9755475,AUI,006104,NDC,NDDF,49452165001
88050,2129,9756761,AUI,008584,NDC,NDDF,00395051392
88051,2129,9756761,AUI,008584,NDC,NDDF,00395051528


In [11]:
print(rxnorm_ndc_df.CUI_CODE.min())
print(rxnorm_ndc_df.CUI_CODE.max())

1952
2285607


In [12]:
write2file(rxnorm_ndc_df,join(processed_map_prefix,"rxnorm_ndc_df"))

In [17]:
norm_ndc = read_data(join(processed_map_prefix,"rxnorm_ndc_df"),
                     dtype={"CUI_CODE":str, "CODE":str})[['CUI_CODE','CODE']].drop_duplicates()
norm_ndc = norm_ndc.groupby(['CUI_CODE'])['CODE'].apply(','.join).reset_index()
# norm_ndc=norm_ndc[['CUI_CODE','NDCs']].drop_duplicates()
norm_ndc.head()

,CUI_CODE,CODE
0,1000000,"65597011530,65597011590,65597011507,6559701151..."
1,1000001,"00093500456,00093500498,13668038505,1366803853..."
2,1000005,"65597011630,65597011690,65597011607,6559701161..."
3,1000006,"00024131302,00024131326,00074121201,0007412130..."
4,1000013,"22840575602,22840575605,54575047602,5457504761..."


In [16]:
norm_ndc['CUI_CODE'].min()

1952

In [18]:
write2file(norm_ndc,join(processed_map_prefix,"rxnorm_ndc_list_df"))

In [10]:
rxnorm_ndc_list_df=read_data(join(processed_map_prefix,"rxnorm_ndc_list_df"))
rxnorm_ndc_list_df["CUI_CODE"].min()

91348

In [11]:
rxnorm_ndc_list_df["CUI_CODE"].max()

2285607

## MAP DISEASE SNOMED_CID -> ICD9

In [1]:
import numpy as np
import pandas as pd
import re
# import re
from os.path import join

from tools import *

# from sklearn.decomposition import LatentDirichletAllocation
# from scipy.spatial import distance

# import matplotlib.pyplot as plt

## rename EXTRACT_FEATURES to DIMENSION_REDUCTION
read_prefix = "/data/MIMIC3/"
mapping_prefix = "/data/liu/mimic3/MAPPING/"

In [6]:
mimic_icd = read_data(join(read_prefix,"D_ICD_DIAGNOSES"),dtype={'ICD9_CODE':str})
mimic_icd.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [7]:
mimic_icd[mimic_icd.ICD9_CODE=="42731"]

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
4461,4462,42731,Atrial fibrillation,Atrial fibrillation


In [23]:
icd9_SNOMED_df2 =pd.read_csv(join(mapping_prefix,'ICD9CM_SNOMED_MAP_1TOM_201912.txt')
                         ,sep='\t', dtype={'ICD9_CODE':str, 'SNOMED_CID':str}).dropna(subset=['SNOMED_CID'])
icd9_SNOMED_df2.head()

,ICD_CODE,ICD_NAME,IS_CURRENT_ICD,IP_USAGE,OP_USAGE,AVG_USAGE,IS_NEC,SNOMED_CID,SNOMED_FSN,IS_1-1MAP,CORE_USAGE,IN_CORE
1,V57.89,Care involving other specified rehabilitation ...,1,1.91603,0.29744,1.106735,1.0,305839006,Seen by arts therapy service (finding),0,NaN,0
2,V57.89,Care involving other specified rehabilitation ...,1,1.91603,0.29744,1.106735,1.0,305840008,Seen by art therapy service (finding),0,NaN,0
3,V57.89,Care involving other specified rehabilitation ...,1,1.91603,0.29744,1.106735,1.0,305844004,Seen by play therapy service (finding),0,NaN,0
4,V57.89,Care involving other specified rehabilitation ...,1,1.91603,0.29744,1.106735,1.0,305842000,Seen by drama therapy service (finding),0,NaN,0
5,V57.89,Care involving other specified rehabilitation ...,1,1.91603,0.29744,1.106735,1.0,305843005,Seen by music therapy service (finding),0,NaN,0


In [10]:
icd9_SNOMED_df =pd.read_csv(join(mapping_prefix,'ICD9CM_SNOMED_MAP_1TO1_201912.txt')
                         ,sep='\t', dtype={'ICD9_CODE':str, 'SNOMED_CID':str}).dropna(subset=['SNOMED_CID'])

icd9_SNOMED_df = icd9_SNOMED_df[["ICD_CODE","SNOMED_CID"]]
icd9_SNOMED_df.head()

,ICD_CODE,SNOMED_CID
0,427.31,49436004
1,599.0,68566005
2,486,233604007
3,585.6,46177005
4,414.01,1641000119107


In [11]:
icd9_SNOMED_df['ICD9_CODE']=icd9_SNOMED_df['ICD_CODE'].apply(lambda x: x.replace(".",""))
icd9_SNOMED_df.head()

,ICD_CODE,SNOMED_CID,ICD9_CODE
0,427.31,49436004,42731
1,599.0,68566005,5990
2,486,233604007,486
3,585.6,46177005,5856
4,414.01,1641000119107,41401


In [13]:
write2file(icd9_SNOMED_df,join(processed_map_prefix,"snomed_icd9_df"))

In [5]:
icd9_SNOMED_df=read_data(join(processed_map_prefix,"snomed_icd9_df"),dtype={"SNOMED_CID":str,"ICD9_CODE":str})
icd9_SNOMED_df.head()

,ICD_CODE,SNOMED_CID,ICD9_CODE
0,427.31,49436004,42731
1,599.0,68566005,5990
2,486,233604007,486
3,585.6,46177005,5856
4,414.01,1641000119107,41401


In [22]:
icd9_SNOMED_map = icd9_SNOMED_df.set_index('SNOMED_CID')['ICD9_CODE'].to_dict()
icd9_SNOMED_map.get("86699002", None)

In [29]:
for folder in ["hospital_course","medications_on_admission","medical_history"]:
    disease_df = read_data(join(save_csv_prefix,folder,"ALL/ALL_Disease")).drop(columns=["index"])
    disease_df["ICD9_CODE"]=disease_df["SNOMEDCT_US"].apply(lambda x: (",").join(
        filter(None,[icd9_SNOMED_map.get(key, None) for key in x.strip().split(',')])))
    write2file(disease_df,join(save_csv_prefix,folder,"ALL/ALL_Disease_icd9"))

In [27]:
disease_df["ICD_CODE"]=disease_df["SNOMEDCT_US"].apply(
    lambda x: [icd9_SNOMED_map.get(key, None) for key in x.strip().split(',')])



disease_df.head()

,SUBJECT_ID,HADM_ID,GROUP_RANK,START INDEX,END INDEX,SEMANTIC,ASSERTION,CUI,sentProb,conceptProb,NE,SNOMEDCT_US,ICD_CODE,ICD9_CODE
0,6702,143782,1,111,127,semantic=problem,assertion=present,cui=C0032227,sentProb=0.825,conceptProb=1.0,ne=pleural effusion,60046008,[5119],5119
1,6702,143782,1,172,186,semantic=problem,assertion=present,cui=C0001418,sentProb=0.524,conceptProb=0.996,ne=adenocarcinoma,"443961001, 35917007","[None, None]",
2,20630,161386,1,211,222,semantic=problem,assertion=absent,cui=C0702249,sentProb=0.992,conceptProb=0.999,ne=infiltrates,231287002,[None],
3,20630,161386,1,226,235,semantic=problem,assertion=absent,cui=C0013687,sentProb=0.992,conceptProb=0.993,ne=effusions,41699000,[None],
4,17496,127002,1,105,113,semantic=problem,assertion=present,cui=C0277797,sentProb=0.677,conceptProb=0.678,ne=afebrile,86699002,[None],


In [16]:
icd9_SNOMED_df[icd9_SNOMED_df['SNOMED_CID']=='4834000']

,ICD_CODE,ICD_NAME,IS_CURRENT_ICD,IP_USAGE,OP_USAGE,AVG_USAGE,IS_NEC,SNOMED_CID,SNOMED_FSN,IS_1-1MAP,CORE_USAGE,IN_CORE
5807,002.0,Typhoid fever,1,0.00004,NaN,0.00002,0,4834000,Typhoid fever (disorder),1,0.0005,1


In [6]:
# concatcsv.SEMANTIC.unique()
medical_history_df=read_data(join(save_csv_prefix,"medical_history/ALL/ALL"))
medical_history_df.head()

,SUBJECT_ID,HADM_ID,GROUP_RANK,START INDEX,END INDEX,SEMANTIC,ASSERTION,CUI,sentProb,conceptProb,NE
0,19981,159283,2,21,39,problem,present,C0018834 SNOMEDCT_US[16331000],0.998,1.0,heartburn symptoms
1,19981,159283,2,72,98,problem,present,0.971,0.971,additional epigastric pain,NaN
2,19981,159283,2,83,93,BDL,epigastric,NaN,NaN,NaN,NaN
3,19981,159283,2,135,146,problem,present,0.994,0.995,a dull ache,NaN
4,19981,159283,2,149,152,test,present,C0079304,0.866,0.995,EGD
